# Algorithm 30: Recycling (Inference)

During inference, AlphaFold2 recycles predictions multiple times to iteratively refine the structure. Each iteration uses the previous output as additional input.

## Algorithm Pseudocode

![RecyclingInference](../imgs/algorithms/RecyclingInference.png)

## Source Code Location
- **File**: `AF2-source-code/model/modules.py`
- **Class**: `AlphaFold`
- **Lines**: 123-200

## Recycling Process

```
Iteration 0: Initial prediction (no recycled info)
     ↓
Iteration 1: Use prev_msa, prev_pair, prev_pos
     ↓
Iteration 2: Further refinement
     ↓
Iteration 3: Final prediction
```

In [ ]:
import numpy as np

np.random.seed(42)

In [ ]:
def recycling_inference(batch, model_fn, num_recycle=3):
    """
    Recycling Inference - Algorithm 30.
    
    Iteratively refines predictions using recycling.
    
    Args:
        batch: Input features
        model_fn: Forward pass function
        num_recycle: Number of recycling iterations
    
    Returns:
        Final prediction
    """
    N_res = batch['aatype'].shape[0]
    
    print(f"Recycling Inference")
    print(f"  Residues: {N_res}")
    print(f"  Recycle iterations: {num_recycle}")
    
    # Initialize recycled tensors
    prev = {
        'prev_pos': np.zeros((N_res, 37, 3)),
        'prev_msa_first_row': np.zeros((N_res, 256)),
        'prev_pair': np.zeros((N_res, N_res, 128)),
    }
    
    # Recycling loop
    for i in range(num_recycle + 1):
        is_final = (i == num_recycle)
        
        print(f"\n  Iteration {i} {'(final)' if is_final else ''}:")
        
        # Embed recycled features
        if i > 0:
            # Add previous outputs to embeddings
            print(f"    Using recycled: prev_pos, prev_msa, prev_pair")
        
        # Forward pass
        output = model_fn(batch, prev, is_final=is_final)
        
        # Update prev for next iteration (if not final)
        if not is_final:
            prev['prev_pos'] = output['final_atom_positions'].copy()
            prev['prev_msa_first_row'] = output['msa_first_row'].copy()
            prev['prev_pair'] = output['pair_repr'].copy()
            print(f"    Updated prev tensors")
        else:
            # Stop gradient on final iteration
            print(f"    Final output (no gradient)")
    
    return output


def dummy_model(batch, prev, is_final=False):
    """Dummy model function for testing."""
    N_res = batch['aatype'].shape[0]
    
    # Simulate prediction (with improvement from recycling)
    noise_scale = 0.5 if is_final else 1.0
    
    return {
        'final_atom_positions': prev['prev_pos'] + np.random.randn(N_res, 37, 3) * noise_scale,
        'msa_first_row': prev['prev_msa_first_row'] + np.random.randn(N_res, 256) * 0.1,
        'pair_repr': prev['prev_pair'] + np.random.randn(N_res, N_res, 128) * 0.1,
    }

In [ ]:
# Test
batch = {
    'aatype': np.random.randint(0, 20, size=64),
}

print("Test Recycling Inference")
print("="*50)

output = recycling_inference(batch, dummy_model, num_recycle=3)

print(f"\nFinal output keys: {list(output.keys())}")

## Source Code Reference

```python
# From AF2-source-code/model/modules.py

class AlphaFold(hk.Module):
  """AlphaFold model with recycling.

  Jumper et al. (2021) Suppl. Alg. 30 "RecyclingInference"
  Jumper et al. (2021) Suppl. Alg. 2 "Inference"
  """

  def __call__(self, batch, is_training, ...):
    # Initialize prev
    prev = _get_prev_init(batch)
    
    # Recycling loop
    for i in range(num_recycle + 1):
      ret = impl(batch_i, prev, is_training=False, ...)
      prev = get_prev(ret)
    
    return ret
```